# Radiantloom Email Assist 7B

In [ ]:
# install required packages
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece langchain gradio sentence-transformers

In [ ]:
# import libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [ ]:
# load the model with 4bit quantization
model_name = "aigeek0x0/radiantloom-email-assist-7b"

def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

# load model
model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

# define stop token ids
stop_token_ids = [0]

In [ ]:
# specifiy the prompt
prompt = """

<s>[INST]
<<SYS>>Given an input, provide a clear and concise summary and action items in less than 50 words.<</SYS>>

Answer the question below from context below :

Convert the summary into a voice memo, keeping it under one minute:

From: deb@example.com
To: tan@xyz.com
Subject: Review and Feedback on Radiantloom Models for Business Integration

Hi Tan,

I hope this email finds you well. We are currently in the process of exploring Radiantloom models for potential integration into our business operations. Specifically, we are interested in your insights on the "radiantloom-email-assist-7b" model.

To ensure that we make well-informed decisions, we kindly request that you take some time to thoroughly review the documentation provided for this particular model. Your expertise and perspective are valuable to us, and we would appreciate any insights or observations you can share regarding the features, capabilities, and potential benefits of the "radiantloom-email-assist-7b" model.

If you come across any noteworthy points, concerns, or suggestions during your review, please don't hesitate to bring them to our attention. Your feedback will play a crucial role in shaping our decision-making process as we assess the suitability of Radiantloom models for our business needs.

Thank you in advance for your time and effort in this matter. We look forward to hearing your thoughts and recommendations.

Best regards,

Deb


[/INST]</s>

"""

In [ ]:
# run inference
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=512,
    temperature=0.1, repetition_penalty=1.1, top_p=0.7, top_k=50)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text)

# Langchain Implementation

In [ ]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# configure huggingface pipiline
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=1024,                        # Increase this upto 4096 with a more powerful GPUs.
        temperature=0.1,
        do_sample=True,
        top_k=50,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
# import libraries
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [ ]:
# function for running the chain

llm = HuggingFacePipeline(pipeline=pipeline)

def email_assist(prompt):

  template = """<s>[INST]
  <<SYS>>
  Given an input, provide a clear and concise summary and action items in less than 50 words.
  <</SYS>>

  Answer the question below from context below:
  {context}
  {question} [/INST] </s>
  """

  question = """Convert the summary into a voice memo, keeping it under one minute:"""


  context = prompt

  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question,"context":context})
  return response

# Gradio Application

In [ ]:
# launch gradio UI for inference

import gradio as gr

def summarizer(text):
    prompt = text
    response = email_assist(prompt)
    return response

pred = gr.Interface(
    summarizer,
    title = "Radiantloom Email Assist - Turn Emails to Voice Memos & Chat Messages",
    inputs=[
        gr.Textbox(
            label="Email Text",
            lines=1,
            placeholder="""

From: deb@example.com
To: tan@xyz.com
Subject: Using Email Assistant Model

Hi Tan,

We are exploring Radiantloom models for our business.
Please review their documentation for the "radiantloom-email-assist-7b model and let me know what you think.

Thanks.

Best,
Deb
            """,
        ),
    ],
    outputs='text',
)

pred.launch(share=True)